In [1]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
%matplotlib inline

In [2]:
emotions = np.load('./data/labeled_emotion.npz',encoding='latin1')
images_train = emotions['input_train']
labels_train = emotions['input_labels']
images_valid = emotions['input_valid']
labels_valid = emotions['input_labels_valid']
labels = emotions['emotions']
print('image_train shape' , images_train.shape)
print('images_valid shape' , images_valid.shape)
print(labels)

In [3]:
N = len(images_train)
BS = 200
minibatchs = int(np.ceil(N/BS))
iterations = 50
imshape = 64*64*3
outputShape = 7

In [4]:
images_train = scale(np.asarray(images_train).reshape(-1,imshape).astype('float64'),axis=1)
images_valid = scale(np.asarray(images_valid).reshape(-1,imshape).astype('float64'),axis=1)

#one-hot encoding
labels_train = OneHotEncoder(sparse=False).fit_transform(labels_train.reshape(-1,1))

#one-hot encoding
labels_valid = OneHotEncoder(sparse=False).fit_transform(labels_valid.reshape(-1,1))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, imshape])
y_ = tf.placeholder(tf.float32, shape=[None, outputShape])

#First Convolutional Layer
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,64,64,3])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#Second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#Densely Connected Layer
W_fc1 = weight_variable([16 * 16 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#drop_out
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#readout_layer
W_fc2 = weight_variable([1024, 7])
b_fc2 = bias_variable([7])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [6]:
for epoch in range(iterations):
    #scramble the indexes:
    ix = np.random.permutation(N)
    input_train = images_train[ix,:]
    input_label = labels_train[ix]
    for i in range(minibatchs):
        x_batch = images_train[i*BS:min(N, BS*(i+1))]
        y_batch = labels_train[i*BS:min(N, BS*(i+1))]
        train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: 0.5})
    if epoch%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:images_train, y_: labels_train, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(epoch, train_accuracy))
        print("test accuracy %g"%accuracy.eval(feed_dict={x: images_valid, y_: labels_valid, keep_prob: 1.0}))

step 0, training accuracy 0.137796
test accuracy 0.137457
step 10, training accuracy 0.617884
test accuracy 0.590574
step 20, training accuracy 0.913633
test accuracy 0.861561
step 30, training accuracy 0.979721
test accuracy 0.918999
step 40, training accuracy 0.997923
test accuracy 0.936181


In [8]:
train_accuracy = accuracy.eval(feed_dict={x:images_train, y_: labels_train, keep_prob: 1.0})
print("step %d, training accuracy %g"%(epoch, train_accuracy))
print("test accuracy %g"%accuracy.eval(feed_dict={x: images_valid, y_: labels_valid, keep_prob: 1.0}))

step 49, training accuracy 1
test accuracy 0.940599


In [9]:
saver.save(sess, 'emotion_iter_'+str(epoch))

'emotion_iter_49'

In [26]:
p_valid = sess.run(y_conv, feed_dict={x: images_valid, keep_prob: 1.0})
label = np.array([0,1,2,3,4,5,6])
result = labels[p_valid.argmax(axis=1)]
confusion_matrix(emotions['input_labels_valid'],p_valid.argmax(axis=1))

array([[273,   4,   3,   2,   7,   7,   0],
       [  1, 264,   2,   5,   2,   1,   1],
       [  0,   1, 286,   7,   1,   9,   0],
       [  0,   1,   5, 393,   4,   1,   2],
       [  1,   0,   3,   5, 213,   2,   8],
       [  3,   2,   8,   1,   4, 225,   0],
       [  0,   3,   1,   4,  10,   0, 262]])

In [45]:
num_pic = 14
cur = 0
count = 0
cur_label = 0
correct = emotions['input_labels_valid']
cur_lab = p_valid.argmax(axis=1)
lst_of_wrong = []
while num_pic > 0:
    if correct[cur] >= cur_label and correct[cur]!= cur_lab[cur]:
        print(str(cur) + '\tcorrect label '+ labels[correct[cur]] + '\tpredicted label ' + labels[int(cur_lab[cur])])
        lst_of_wrong.append(cur)
        count = count + 1
        num_pic = num_pic - 1
        if count == 2:
            count = 0
            cur_label = cur_label + 1
    cur = cur + 1

3	correct label angry	predicted label neutral
21	correct label angry	predicted label happy
326	correct label happy	predicted label sad
353	correct label happy	predicted label neutral
572	correct label sad	predicted label happy
668	correct label sad	predicted label neutral
985	correct label neutral	predicted label disgust
990	correct label neutral	predicted label sad
1290	correct label fear	predicted label sad
1310	correct label fear	predicted label neutral
1518	correct label disgust	predicted label angry
1521	correct label disgust	predicted label happy
1786	correct label surprise	predicted label fear
1789	correct label surprise	predicted label fear


/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
def format_image(image):
    size = 64, 64
    pic = Image.fromarray(image.astype('uint8'), 'RGB')
    pic = pic.resize(size)
    arr = np.array(pic,dtype='f')
    arr = scale(arr.reshape(-1,64*64*3).astype('float64'),axis=1)
    return arr

In [46]:
valid = emotions['input_valid']

In [64]:
im=Image.fromarray(valid[1789].astype('uint8'), 'RGB')
plt.imshow(im)

In [66]:
num_pic = 7
cur = 0
count = 0
cur_label = 0
correct = emotions['input_labels_valid']
cur_lab = p_valid.argmax(axis=1)
lst_of_wrong = []
while num_pic > 0:
    if correct[cur] >= cur_label and correct[cur] == cur_lab[cur]:
        print(str(cur) + '\tcorrect label '+ labels[correct[cur]] + '\tpredicted label ' + labels[int(cur_lab[cur])])
        lst_of_wrong.append(cur)
        count = count + 1
        num_pic = num_pic - 1
        if count == 1:
            count = 0
            cur_label = cur_label + 1
    cur = cur + 1

0	correct label angry	predicted label angry
296	correct label happy	predicted label happy
573	correct label sad	predicted label sad
876	correct label neutral	predicted label neutral
1282	correct label fear	predicted label fear
1514	correct label disgust	predicted label disgust
1757	correct label surprise	predicted label surprise


/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [74]:
im=Image.fromarray(valid[1757].astype('uint8'), 'RGB')
plt.imshow(im)